In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
path_img="/kaggle/input/flicker8k/data/Images/"
path_cap="/kaggle/input/flicker8k/data/captions.txt"

In [9]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from tensorflow.keras.layers import *

In [10]:
df=pd.read_csv(path_cap)
df.head()

# Adding __start__ and __end__ to data 

In [11]:
import time

i=0
k=1
info={}
img_name=[]
try:
    while True:
        info[df.iloc[i,0]] = ["<start> "+df.iloc[i,1]+" <end>", 
                              "<start> "+df.iloc[i+1,1]+" <end>",
                              "<start> "+df.iloc[i+2,1]+" <end>",
                              "<start> "+df.iloc[i+3,1]+" <end>",
                              "<start> "+df.iloc[i+4,1]+" <end>"]
        img_name.append(df.iloc[i,0])
        i=i+5
        k+=1
        
        if k == 1701:
            break
except:
    print("The data has been processed")

In [12]:
# import time
# while True:
#     print("5")
#     time.sleep(60)
    

In [13]:
len(info)

In [14]:
len(img_name)

In [15]:
print(img_name[0])
info[img_name[0]]

# Image captions and datapresent

In [16]:
import matplotlib.pyplot as plt
from IPython.display import Image,display
from PIL import Image

In [17]:
k=0
z=Image.open(path_img+img_name[k])
display(z)
try:
    for i in range(5):
        print(info[img_name[k]][i])
except :
    print("Error")
    pass

# Cerating a Corpus of words

In [18]:
cap_list=[]
val=0
try:
    while True:
        for i in range(5):
            cap_list.append(info[img_name[val]][i])
        val+=1
except :
    print("Done")

In [19]:
len(cap_list)

In [20]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [21]:
vocab_size = 18000
embedding_dim = 16
trunc_type='post'
pad_type="post"
oov_tok = "<OOV>"
max_length=40

In [22]:
tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(cap_list)

In [23]:
word_index=tokenizer.word_index
vocab_size=len(word_index)
vocab_size

## Finding the best maxlength paramerter

In [24]:
lengths=[len(t.split()) for t in cap_list]

In [25]:
plt.figure(figsize=(10,5))
plt.hist(lengths,bins=len(set(lengths)))
plt.tight_layout()
plt.show()

# Preprocessing the data -- Sequencing the data

In [26]:
def get_sequences(tokenizer,cap_list):
    sequences=tokenizer.texts_to_sequences(cap_list)
    #padded=pad_sequences(sequences,maxlen=max_length,truncating=trunc_type,padding=pad_type)
    return sequences

In [27]:
val=0
padded_cap_seq=[]
try:
    while True:
        padded_cap_seq.append(get_sequences(tokenizer,info[img_name[val]]))
        val+=1
except:
    print("Done")

In [28]:
info[img_name[0]]

In [29]:
padded_cap_seq[0]

In [30]:
## Checking of reverse conversion
tokenizer.sequences_to_texts(padded_cap_seq[0])

# Preprocessing the Image
## opening the image file and setting the input shape to (224,224,3)

In [31]:
## convert the image data to array
def img_generator(img_path):
    
    img_arr=Image.open(img_path)
    img_arr=img_arr.resize((224,224))
    #img_arr=tf.keras.preprocessing.image.img_to_array(img_arr)/255
    return np.expand_dims(img_arr, axis=0)

img_generator(path_img+img_name[0]).shape
img_generator(path_img+img_name[0])

# Mapping the padded seq of text as it is the only way to get the captions for each image. 

In [32]:
len(img_name)

In [33]:
info_seq={}
i=0
#j=0
try:
    while True:
        info_seq[img_name[i]]=padded_cap_seq[i]
        #print(i)
        i+=1
        #j+=1
except:
    print(f"error : {i}")

In [34]:
print(img_name[0])
padded_cap_seq[0]

# Building the model using ImceptionV3

In [35]:
from tensorflow.keras.applications import VGG16
modele = VGG16(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling='avg')
modele.summary()

In [36]:
nn_img={}
i=0
from tqdm import tqdm

try:
    for i in tqdm(range(len(img_name))):
        img_arr=img_generator(path_img+img_name[i])
        pred=modele.predict(img_arr).reshape(512)
        nn_img[img_name[i]]=pred
        
        pass
except:
    print("Done")

In [37]:
import pickle

In [38]:
with open("process_img.p","wb") as pickle_f:
    pickle.dump(nn_img, pickle_f)

In [39]:
## Opening the predicted file
with open("./process_img.p","rb") as f:
    nn_img=pickle.load(f,encoding="bytes")

In [40]:
nn_img

In [41]:
len(nn_img)

## Splitting each word and creating a variable out of it so that the next word works as the word we need to predict

## Create 3 variables 
- ### 1-image: holds image features
- ### 2-prev_word
- ### 3-next_word: to be used as the word needed to be predicted

In [42]:
from tensorflow.keras.utils import to_categorical

In [43]:
info_seq[img_name[0]]

In [44]:
nn_img[img_name[0]]

In [45]:
## input is photo dic (nn_img[]), caption - seq (info_seq[])dict already defined
## img is the index value

def generator(photo,caption):
    X=[]
    y_in=[]
    y_out=[]
    # loop 1: for appending the images
    for k,vv in caption.items():
        # loop 2: For the 5 different captions in a single image
        for v in vv:
            # loop3 : For a single caption out the 5 
            for i in range(1,len(v)):
                # appending the image to X
                X.append(photo[k])
                # taking the first part of the sequence
                in_seq=[v[:i]]
                # the park of sequence the model should predict next
                out_seq=v[i]
                # Now making the array even for processing by padding
                
                in_seq=pad_sequences(in_seq,maxlen=max_length,padding=pad_type,truncating=trunc_type)[0]
                # As the y_test is going to be 1 in length to_categrcal converts it to vocab size
                out_seq=to_categorical([out_seq],num_classes=vocab_size+1)[0]

                y_in.append(in_seq)
                y_out.append(out_seq)
    
    return X,y_in,y_out
                

In [46]:
X,y_in,y_out=generator(nn_img,info_seq)

In [47]:
len(X),len(y_in),len(y_out)

In [49]:
y_tensor_in=tf.constant(y_in)
y_tensor_in

In [ ]:
y_tensor_out=df.co

In [48]:
X_tensor=tf.constant(X)
X_tensor

In [41]:
X=np.array(X)
y_in=np.array(y_in,dtype="float64")
y_out=np.array(y_out,dtype="float64")

In [42]:
X.shape,y_in.shape,y_out.shape

# Now creating NN layer for imges and captions(CNN and LSTM) and concatinating them 

In [43]:
embedding_size=128
image_model=tf.keras.Sequential()
image_model.add(Dense(embedding_size,input_shape=(512,),activation="relu"))
image_model.add(RepeatVector(max_length))

image_model.summary()

In [44]:
language_model=tf.keras.Sequential()
language_model.add(Embedding(input_dim=vocab_size,output_dim=embedding_size,input_length=max_length))
language_model.add(LSTM(256,return_sequences=True))
language_model.add(TimeDistributed(Dense(embedding_size)))

language_model.summary()

In [45]:
conca=Concatenate()([image_model.output,language_model.output])
x=LSTM(128,return_sequences=True)(conca)
x=LSTM(512,return_sequences=False)(x)
x=Dense(vocab_size+1)(x)
out=Activation('Softmax')(x)

model=tf.keras.Model(inputs=[image_model.input,language_model.input],outputs=out)
model.compile(loss="categorical_crossentropy",optimizer='RMSprop',metrics=["accuracy"])
model.summary()

In [46]:
from tensorflow.keras.utils import plot_model
plot_model(model,to_file="./model_plot.png",show_shapes=True,show_layer_names=True)

In [47]:
from tensorflow.keras.callbacks import ModelCheckpoint
mc = ModelCheckpoint('best_model_acc.h5', monitor = 'accuracy' , mode = 'max', verbose = 1 , save_best_only = True)

In [48]:
model.fit([X,y_in],y_out,batch_size=512,epochs=50,callbacks=[mc])

In [49]:
def getImage(file_path):
    test_img=Image.open(file_path)
    test_img=test_img.resize((224,224))
    #test_img=tf.keras.preprocessing.image.img_to_array(test_img)/255
    test_img=np.expand_dims(test_img,axis=0)
    return test_img

In [55]:
test_feature=modele.predict(getImage(path_img+img_name[1]))

In [56]:
test_feature

In [57]:
text_inp=["start"]
count=0
caption=" "
while count<25:
    count+=1
    encoded=[]
    for i in text_inp:
        encoded.append(tokenizer.word_docs[i])
    encoded=[encoded]
    encoded=pad_sequences(encoded,padding=pad_type,truncating=trunc_type,maxlen=max_length)
    
    prediction=np.argmax(model.predict([test_feature,encoded]))
    
    sampled_word=tokenizer.index_word[prediction]
    caption=caption+" "+ sampled_word
    if sampled_word=="end":
        break
    text_inp.append(sampled_word)

In [58]:
" ".join(text_inp[1:])

In [59]:
Image.open(path_img+img_name[1])

In [64]:
# import time 
# while True:
#     print("5")
#     time.sleep(60)

# Saving the Tokenizer for deploying

In [61]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [62]:
# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)